In [ ]:
# Imports
import neurods
import numpy as np
import cortex
import os
import matplotlib.pyplot as plt
# Configure defaults for plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'
%matplotlib inline

In the lecture, we saw how we can use regression to estimate the weight that each of our inputs x contributes to the output y. We also saw that we can estimate a confidence interval for the different weights using the bootstrap algorithm.

In this homework, we will use the bootstrap algorithm to:
- establish a confidence interval for a mean of n numbers
- establish a confidence interval for the weights of a linear model with multiple outputs, specifically, 40000 voxels.

## Confidence intervals for the means of n numbers:

Use the cells below to load a set of 50 numbers that were randomly generated for you.


In [ ]:
url_file = 'https://www.dropbox.com/s/idxsmsxvugwanjl/lab10_data.npy'
neurods.io.download_file(url_file, 'lab10_data.npy',
                         root_destination=os.path.abspath(os.curdir),
                         replace = True)
numbers = np.load('lab10_data.npy')

We are interested in obtaining the mean of the distribution that generated those numbers. 

We use the cell below to estimate the mean of that distribution and print it.

In [ ]:
mu = numbers.mean()
print(mu)

We have 50 points from the mystery distribution. We can't be sure of how much to trust the mean estimated above. 

We will use those same 50 points to derive a confidence interval, by running a bootstrap test. 

The bootstrap algorithm estimates an empirical distribution for the estimated variable of interest. Empirical here means that the distribution is estimated from the data. You can visualise an empirical distribution of a variable by contructing a histogram of the different values this variable can take. Because of noise, estimating a parameter has an inherent variance, or error, and the boostrap algorithm allows us to estimate that variance.

To contruct the empirical distribution of a statistic $s$, the algorithm will run a large number of iterations $M$. A statistic is any quantity that is derived from a dataset. In this first example, the statistic is the mean of all the points. In other examples, the statistic can be the regression weight of a variable, or the difference between the regression weights of two variables.

At each iteration $i$, the algorithm:
- samples n points with replacement from the n original points.
- estimates the statistic $s^i$ using these sampled points.

The set $\{s^1,s^2...s^M \}$ is used as an empirical distribution, from which the interval of confidence is estimated.

This is how you can sample with replacement in python:

In [ ]:
original_points = np.array([14,1,7,8,10,5,4,13])

print('original points are: {}'.format(original_points))

n = original_points.shape[0]

sampled_index = np.random.choice(n,n)

sampled_set = original_points[sampled_index]

print('one sampled set of n points with replacement is: {}'.format(sampled_set))


Run the above cell until you are convinced that it's working correctly. 

#### 1- Now, you will construct a function that uses the sampling method above to estimate the bootstrap samples.

a- use the space below to write a function that:
- takes a one dimentional array as input
- computes the length of that array
- samples n indices with replacement from 1...n
- uses these indices to construct a new sample of data
- return the mean of that sample.

In [ ]:
def randomize_mean(X):
### STUDENT ANSWER
    n = X.shape[0]
    sampled_index = np.random.choice(n,n)
    sampled_X = X[sampled_index]
    return sampled_X.mean()

b- use the function you defined above to sample 2000 means of the original `numbers` array.
- before you start you should define an array `sample_means` of size (2000,1) which you will use to store the 2000 samples. This is the same kind of preallocation we saw in class. For each iteration i, you will need to modify the entry `sample_means[i,0]`.
- plot a histogram of the 2000 sample means.

In [ ]:
### STUDENT ANSWER
sample_means = np.zeros((2000,1))

for i in np.arange(2000):
    sample_means[i,0] = randomize_mean(numbers)
    
plt.hist(sample_means)

c - From the plot above, name a reasonable interval that contains most of the sampled means. 

In [ ]:
# write the interval as (some number, some other number)
### STUDENT ANSWER
#(6.6,7.1) OR
#(6.5,7.3) OR
#(6.7, 7.05)
# etc

The distribution above will actually give us a confidence level in our mean. For example, from the histogram you plotted, you should feel pretty confident that the mean of the distribution that the numbers were sample from is not likely to be 0 or smaller than 0. However, you might not feel confident about whether the true mean is more likely to be closer to 6.8 than to 6.86.

We can use the distribution above to construct a confidence interval. We can use the level of confidence that we want. If we build a 95% of confidence, and say that we are 95% confidence that the true mean lies within that interval, then that means if we repeated the entire analysis an infinite number of time (i.e. getting a new set of points each time) then 95% of those constructed intervals will contain the true mean.

How do we construct a 95% confidence interval (or 98% or 99% etc)? We can look at our empirical distribution and build an interval that contains 95% of the sampled means. We can exclude the bottom 2.5% and the top 2.5% of the sampled means. This will give us the range of 95% of the sampled means.

How to we use numpy to compute these percentiles? By simply using the function `np.percentile`. The second argument is the percentile you want (in percents):

In [ ]:
percentile_values = [2.5, 10, 20, 50, 80, 90, 97.5]

for p in percentile_values:
    perc = np.percentile(sample_means, p)
    print('the {} percentile of the data is {}'.format(p, perc) )
    

The interval between the 10th and the 90th percentile of the data is an 80% confidence interval, the interval between the 20th and the 80th percentile of the data is an 60% confidence interval,  etc. 

Let's plot some confidence intervals on the histogram of `sample_means`. First, we will define below a function that plots a confidence interval as a rectangle. The function takes as input:
- the axis of the current plot
- the lower bound of the interval
- the upper bound
- the elevation of the rectangle on the y-axis
- the color of the rectangle
- a string label for the name of the confidence interval.

In [ ]:
from matplotlib.patches import Rectangle 

def draw_interval(ax, lower_bound, upper_bound, y_position, color = 'red', name = ''  ):
    height = 2
    width = upper_bound - lower_bound
    start_coordinate = (lower_bound,y_position) 
    rect = Rectangle(start_coordinate, width, height, color = color, label = name)
    ax.add_patch(rect)

Below, you can see how this function is used to draw a 80% confidence interval:

In [ ]:
# initialize figure
fig = plt.figure()
ax = fig.add_subplot(111)
# add the histogram
ax.hist(sample_means,100)

# define the parameters of the confidence interval
lower_bound = np.percentile(sample_means, 10)
upper_bound = np.percentile(sample_means, 90)
y_position = 2
name = '80% confidence interval'
color = 'red'

# draw the confidence interval
draw_interval(ax, lower_bound, upper_bound, y_position, color = color, name = name  )


# add legend
ax.legend();

d - use the cell below to plot another histogram.  
- You should modify the code of the cell above so that it show simultaneously a 80%, 90%, 95% and a 99% confidence interval (you should call the draw_interval function 4 times, each time with different parameters).
- You need to pick a different y_position for each confidence interval so that they don't overlap. You need to give it a different name and a different color. It's a good idea to put the smaller intervals at the top of the larger ones, but you are free to pick the locations you want.
- At the end you should have only one histogram with all four confidence interval. It should be easy to distinguish the different intervals from each other using the legend.  

In [ ]:
### STUDENT ANSWER

# initialize figure
fig = plt.figure()
ax = fig.add_subplot(111)
# add the histogram
ax.hist(sample_means,100)

# define the parameters of the confidence interval
lower_bound = np.percentile(sample_means, 10)
upper_bound = np.percentile(sample_means, 90)
y_position = 20
name = '80% confidence interval'
color = 'red'

# draw the confidence interval
draw_interval(ax, lower_bound, upper_bound, y_position, color = color, name = name  )

# define the parameters of the confidence interval
lower_bound = np.percentile(sample_means, 5)
upper_bound = np.percentile(sample_means, 95)
y_position = 15
name = '90% confidence interval'
color = 'blue'

# draw the confidence interval
draw_interval(ax, lower_bound, upper_bound, y_position, color = color, name = name  )


# define the parameters of the confidence interval
lower_bound = np.percentile(sample_means, 2.5)
upper_bound = np.percentile(sample_means, 97.5)
y_position = 10
name = '95% confidence interval'
color = 'yellow'

# draw the confidence interval
draw_interval(ax, lower_bound, upper_bound, y_position, color = color, name = name  )


# define the parameters of the confidence interval
lower_bound = np.percentile(sample_means, 0.5)
upper_bound = np.percentile(sample_means, 99.5)
y_position = 5
name = '99% confidence interval'
color = 'cyan'

# draw the confidence interval
draw_interval(ax, lower_bound, upper_bound, y_position, color = color, name = name  )

# add legend
ax.legend();

## Confidence intervals for the difference between two regression weights.

The bootstrap algorithm can be used to construct confidence intervals for other statistics, for example it can be used to estimate the estimation variance of a regression weight, or even the difference between two weights. In class, we saw how we can use the bootstrap to compute a confidence interval for the difference between the real price of oranges and apples.

A very common setting in fMRI is to compare how the brain responds to two conditions, and find regions that respond more to one condition than to another condition. We saw in class how we can use regression to estimate the response of each voxel in the brain to different conditions. In our case, we had 5 different conditions. What the OLS function returned was a set of 5 different weights (one for each condition) for every voxel in the brain. These correspond to the estimate response of each voxel to the 5 conditions.

We want to find which voxels respond more to faces than to bodies. We can subtract the weights of bodies from the weights of faces and get a difference map. We can see where the map is positive. This is not a satisfactory answer since it doesn't give us a measure of certainty. We will use bootstrapping here to construct that measure.

Let's reload the data, and recompute the weights:

In [ ]:
from numpy.linalg import inv
def OLS(X,Y):
    first_part = inv(np.dot(X.T, X))
    second_part = np.dot(X.T,Y)
    OLS_solution = np.dot(first_part, second_part)
    return OLS_solution

In [ ]:
basedir = os.path.join(neurods.io.data_list['fmri'],'categories')
design = np.load(os.path.join(basedir,'experiment_design.npz'))
print('Experiment design variables: ', design.keys())
conditions = design['conditions'].tolist()
print('Conditions: ', conditions)

In [ ]:
fmri_files1 = ['s01_categories_{:02d}.nii.gz'.format(run) for run in [1,2]]
fmri_files1 = [os.path.join(neurods.io.data_list['fmri'], 'categories', f) for f in fmri_files1]

sub, xfm = 's01', 'catloc'
cortical_voxels = cortex.db.get_mask(sub, xfm, type='cortical')
# fmri responses:
Y = np.vstack( neurods.io.load_fmri_data(fmri_files1[i], mask=cortical_voxels, do_zscore=True, dtype=np.float32)
              for i in [0,1])
# stimuli:
X = np.vstack([design[run] for run in ['run1','run2']])

We need to first build a design matrix that accounts for the hemodynamic response:

In [ ]:
from neurods.fmri import hrf as generate_hrf
t_hrf, hrf_1 = generate_hrf(tr=2)
n, d = X.shape

conv_X = np.zeros_like(X)
for i in range(d):
    conv_X[:,i] = np.convolve(X[:,i], hrf_1)[:n]

We find below the response of all the voxels in the brain to these 5 different conditions.

In [ ]:
weights = OLS(conv_X, Y)
print('shape of weights is {}'.format(weights.shape))

vol = cortex.Volume(weights[1] - weights[0], sub, xfm, mask = cortical_voxels,vmin = -1.5, vmax = 1.5)
__  = cortex.quickflat.make_figure(vol)
plt.suptitle('faces - bodies', fontsize = 30)
cortex.webshow(vol)

What we need to do to perform a bootstrap test is to resample the data, compute the OLS weights, and then take the difference between the face weight and the body weight (at each voxel). We repeat this a large number of times, and then contruct a confidence interval (at each voxel). 

There is a slight complexity with fMRI data, different fMRI data points are not IID (independent and identically distributed) because of the slow dynamics leading to the signal. By sampling individual points, the dependencies between the consecutive time points will be broken, and therefore the estimated confidence intervals will not be characteristic of the true variance.

Therefore the test we used above will not be adequate to use. We will use here a slightly modified test that samples blocks of data (5TRs) instead of individual TRs. It is implemented below:

In [ ]:
def randomize_OLS_for_fMRI(X,Y):
    n = X.shape[0]
    # the aim is to divide the time course into blocks of 5 TRs and 
    # sample blocks from that set with replacement
    # the number of such blocks is:
    n_blocks = int(n/5)
    # pick n_blocks 5TR blocks with replacement:
    sample_index = np.random.choice(n_blocks,n_blocks)
    # matrix manipulation to allow the selection of blocks, basically
    # this creates an array with each columns corresponding to a block
    # and the different rows correspond to index of the 5 TR in that block 
    block_index = np.arange(n).reshape([n_blocks,-1])
    # now construct the new index by selecting the blocks that were sampled 
    sample_index = block_index[sample_index].reshape([-1])
    # run OLS on the selected data:
    weights = OLS(X[sample_index], Y[sample_index])
    return weights

Now we can use the code below to run the bootstrap: for each iteration, we store the difference between the weight for faces and the weight for bodies. Here we will draw only 100 samples because of limited memory. In a real experiment, we want to draw many more samples.

In [ ]:
n_bootstrap = 100
difference_bootstrap = np.zeros((n_bootstrap,Y.shape[1]))

for i in np.arange(n_bootstrap):
    tmp = randomize_OLS_for_fMRI(X,Y)
    difference_bootstrap[i,:] = tmp[1,:] - tmp[0,:]
print(difference_bootstrap.shape)

2- Use the confidence interval as a test:

We want to use the empirical distribution that we obtained to contruct a 95% confidence interval for the difference of the response for faces and the response for bodies at every voxel. Confidence interval can be used to perform a statistical hypothesis test: we are interested to know which voxels have a different response for faces than for bodies. This means that the difference should be (reliably) different than 0. If 0 is outside of the 95% confidence interval, that means that with 95% certainty the responses for faces and bodies is different. 

This is called a two-sided test because we are simultaneouly looking for regions for which faces have a higher response, and regions for which bodies have a higher response. If 0 is below the lower bound for the 95% confidence interval, then this suggest that the response for faces is larger than the response for bodies. If 0 is above the 95% confidence interval, then this suggest that the response for faces is larger than the response for bodies.

a - Find the lower bound and the upper bound of the 95% confidence intervals for all voxels:
- use np.percentile to find the lower and upper bounds. 
- Look at the parameters of the np.percentile function, you can actually specify which axis to use it on. In one call to np.percentile, you should be able to compute the lower bound and upper bound of the confidence interval.

In [ ]:
# call your variables confidence_interval_lower_bound and confidence_interval_upper_bound
### STUDENT ANSWER
confidence_interval_lower_bound = np.percentile(difference_bootstrap, 2.5, axis = 0)
confidence_interval_upper_bound = np.percentile(difference_bootstrap, 97.5, axis = 0)


The cell below show the regions for which the response for faces seems to be higher (0 is lower than the confidence interval):

In [ ]:
vol = cortex.Volume((confidence_interval_lower_bound>0)*1.0, sub, xfm, mask = cortical_voxels, vmin = -1,
                   vmax = 1)
__  = cortex.quickflat.make_figure(vol, colorbar_ticks=[-1,0,1])
plt.suptitle('faces > bodies', fontsize = 30)

The cell below show the regions for which the response for bodies seems to be higher (0 is higher than the confidence interval):

In [ ]:
vol = cortex.Volume((confidence_interval_upper_bound<0)*1.0, sub, xfm, mask = cortical_voxels, vmin = -1,
                   vmax = 1)
__  = cortex.quickflat.make_figure(vol, colorbar_ticks=[-1,0,1])
plt.suptitle('faces < bodies', fontsize = 30)

ATTENTION: we are not ready yet to make conclusions about brain representations. It is too early to declare the regions we have found as "significantly" more responsive to faces or to bodies. We still have to make important statistical corrections that we will learn about next week.
